In [1]:
import numpy as np

from QC_DFT.NeuralNetwork import QCDFTModel
from QC_DFT.Circuit import QCDFT_Circuit, NN_QCDFT_Circuit
from QC_DFT.Utils import Gates
import pennylane as qml

from tqdm import tqdm

import random
import copy

In [ ]:
def generate_random_gates(n_qubits, gate_count):
    gates = ["X", "Y", "Z", "H", "CX", "CX", "CX", "CX", "CX", "CX", "CX", "CX", "RX", "RY", "RZ"]
    qubits = list(range(n_qubits))

    res = []

    for i in range(gate_count):
        gate = random.choice(gates)
        qubit = random.choice(qubits)
        param = None
        if gate == "CX":
            qubit2 = None
            while qubit2 == None or qubit2 == qubit:
                qubit2 = random.choice(qubits)
            qubit = (qubit, qubit2)
        if gate == "RX" or gate == "RY" or gate == "RZ":
            param = random.uniform(0, 2 * np.pi)
        res += [(gate, qubit, param)]
    
    return res

# gates = (gate(string), qubit, params)
def apply_gate_circuit(circuit, gate):
    gate, qubit, params = gate
    if gate == "X":
        circuit.x(qubit)
    if gate == "Y":
        circuit.y(qubit)
    if gate == "Z":
        circuit.z(qubit)
    if gate == "H":
        circuit.h(qubit)
    if gate == "I":
        circuit.i(qubit)
    if gate == "RX":
        circuit.rx(params, qubit)
    if gate == "RY":
        circuit.ry(params, qubit)
    if gate == "RZ":
        circuit.rz(params, qubit)
    if gate == "CX":
        circuit.cx(*qubit)

    return circuit

def apply_gate_exact(state, gate):
    n_qubits = int(np.log2(state.shape[0]))
    gate, qubit, params = gate

    unitary = None
    if gate == "X":
        unitary = Gates.X()
    if gate == "Y":
        unitary = Gates.Y()
    if gate == "Z":
        unitary = Gates.Z()
    if gate == "H":
        unitary = Gates.H()
    if gate == "I":
        unitary = Gates.I()
    if gate == "RX":
        unitary = Gates.RX(params)
    if gate == "RY":
        unitary = Gates.RY(params)
    if gate == "RZ":
        unitary = Gates.RZ(params)

    if unitary is not None:
        for _ in range(qubit):
            unitary = np.kron(Gates.I(), unitary)
        for _ in range(qubit + 1, n_qubits):
            unitary = np.kron(unitary, Gates.I())

    if gate == "CX":
        unitary = np.zeros((2 ** n_qubits, 2 ** n_qubits), dtype=complex)
        for i in range(2 ** n_qubits):
            if i & (2 ** qubit[0]) != 0:
                unitary[i][i ^ (2 ** qubit[1])] = 1
            else:
                unitary[i][i] = 1

    return unitary @ state @ np.transpose(np.conjugate(unitary))

def get_sqp_exact(state):
    n_qubits = int(np.log2(state.shape[0]))

    sqps = [0.0 for _ in range(n_qubits)]

    for i in range(2 ** n_qubits):
        for j in range(n_qubits):
            if i & (2 ** j) != 0:
                sqps[j] += state[i][i] 

    return np.array(sqps, dtype=float)


def get_fidelity(state, rdms):
    n_qubits = int(np.log2(state.shape[0]))
    trace_list = list(range(n_qubits))
    fidelity = []
    for i in range(n_qubits):
        trace_list_ = copy.deepcopy(trace_list)
        trace_list_.remove(i)
        fidelity += [qml.math.fidelity(qml.math.partial_trace(state, trace_list_), rdms[i])]
    return np.array(fidelity)

In [3]:
nn_models = QCDFTModel.load("Models/default")

In [ ]:
print(nn_models.control_nn)
print(nn_models.target_nn)

In [ ]:
n_qubits = 5
gate_count = 100
iteration_count = 300

In [ ]:
exact_sqp_mean = []
bernardi_sqp_mean = []
proposed_sqp_mean = []

bernardi_sqp_error = []
proposed_sqp_error = []

bernardi_fidelity_mean = []
proposed_fidelity_mean = []

for iteration in tqdm(range(iteration_count)):

    exact_sqp_mean_inner = []
    bernardi_sqp_mean_inner = []
    proposed_sqp_mean_inner = []

    bernardi_sqp_error_inner = []
    proposed_sqp_error_inner = []

    bernardi_fidelity_mean_inner = []
    proposed_fidelity_mean_inner = []


    exact_state = np.zeros((2 ** n_qubits, 2 ** n_qubits), dtype=complex)
    exact_state[0][0] = 1
    bernardi_circuit = QCDFT_Circuit(n_qubits)
    proposed_circuit = NN_QCDFT_Circuit(n_qubits, nn_models)

    gates = generate_random_gates(n_qubits, gate_count)
    for gate in gates:
        bernardi_circuit = apply_gate_circuit(bernardi_circuit, gate)
        proposed_circuit = apply_gate_circuit(proposed_circuit, gate)

    for operator_iterator in range(gate_count):
        exact_state = apply_gate_exact(exact_state, gates[operator_iterator])
        bernardi_state = bernardi_circuit.evolve(1)
        proposed_state = proposed_circuit.evolve(1)

        exact_sqp = get_sqp_exact(exact_state)
        bernardi_sqp = np.real(bernardi_state[:,1,1])
        proposed_sqp = np.real(proposed_state[:,1,1])

        exact_sqp_mean_inner += [np.mean(exact_sqp)]
        bernardi_sqp_mean_inner += [np.mean(bernardi_sqp)]
        proposed_sqp_mean_inner += [np.mean(proposed_sqp)]

        bernardi_sqp_error_inner += [np.sqrt(np.mean(np.square(exact_sqp - bernardi_sqp)))]
        proposed_sqp_error_inner += [np.sqrt(np.mean(np.square(exact_sqp - proposed_sqp)))]

        bernardi_fidelity_mean_inner += [np.mean(get_fidelity(exact_state, bernardi_state))]
        proposed_fidelity_mean_inner += [np.mean(get_fidelity(exact_state, proposed_state))]
                

    exact_sqp_mean += [exact_sqp_mean_inner]
    bernardi_sqp_mean += [bernardi_sqp_mean_inner]
    proposed_sqp_mean += [proposed_sqp_mean_inner]
    bernardi_sqp_error += [bernardi_sqp_error_inner]
    proposed_sqp_error += [proposed_sqp_error_inner]
    bernardi_fidelity_mean += [bernardi_fidelity_mean_inner]
    proposed_fidelity_mean += [proposed_fidelity_mean_inner]

exact_sqp_mean = np.mean(np.array(exact_sqp_mean, dtype=float), axis=0)
bernardi_sqp_mean = np.mean(np.array(bernardi_sqp_mean, dtype=float), axis=0)
proposed_sqp_mean = np.mean(np.array(proposed_sqp_mean, dtype=float), axis=0)

bernardi_sqp_error = np.mean(np.array(bernardi_sqp_error, dtype=float), axis=0)
proposed_sqp_error = np.mean(np.array(proposed_sqp_error, dtype=float), axis=0)

bernardi_fidelity_mean = np.mean(np.array(bernardi_fidelity_mean, dtype=float), axis=0)
proposed_fidelity_mean = np.mean(np.array(proposed_fidelity_mean, dtype=float), axis=0)

In [7]:
folder = "Results"
suffix = ""

np.save(f"{folder}/exact_sqp_mean{suffix}", exact_sqp_mean)
np.save(f"{folder}/bernardi_sqp_mean{suffix}", bernardi_sqp_mean)
np.save(f"{folder}/proposed_sqp_mean{suffix}", proposed_sqp_mean)
np.save(f"{folder}/bernardi_sqp_error{suffix}", bernardi_sqp_error)
np.save(f"{folder}/proposed_sqp_error{suffix}", proposed_sqp_error)
np.save(f"{folder}/bernardi_fidelity_mean{suffix}", bernardi_fidelity_mean)
np.save(f"{folder}/proposed_fidelity_mean{suffix}", proposed_fidelity_mean)